In [4]:
#zero-shot extraction
import os
import pdfplumber
from langchain_openai import ChatOpenAI
from openai import OpenAI

api_key = os.getenv("OPENAI_API_KEY")
# 从PDF文件中提取文本
def extract_text_from_pdf(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text

# 使用模型直接提取信息
def extract_info_with_prompting(text):
    llm = ChatOpenAI(temperature=0, model="gpt-4o-mini", openai_api_key=api_key)
    
    prompt = f"""
    Extract the following epidemiological data from the given text:
    - Reproduction number
    - Growth rate
    - Dispersion parameter
    - Pathogen Transmission
    - Zoonotic/primary infection
    - Secondary transmission
    - Case-fatality ratio (CFR)
    - Infection-fatality ratio (IFR)
    - Attack rate
    - Symptom onset date
    - Onset-admission delay
    - Nosocomial infection
    - Onset-hospitalisation delay
    - Onset-death delay
    - Incubation period
    - Generation time
    - Serial interval
    - Infectious period
    - Clearance rate
    - Recovery rate
    - Transmission chain
    - Seroprevalence
    - Susceptibles
    - Linelist
    - Pandemic
    - Epidemic
    - Outbreak

    Text:
    {text}
    """

    response = llm(prompt)
    return response

# 主函数
def main(pdf_path):
    # 提取PDF文本
    text = extract_text_from_pdf(pdf_path)
    
    # 使用prompting提取信息
    extracted_info = extract_info_with_prompting(text)
    
    # 显示提取的信息
    print(extracted_info)

# 调用主函数，传入PDF文件路径
pdf_path = 'Chains-of-transmission-and-control-of-Ebola-virus-.pdf'

main(pdf_path)

content='Based on the provided text, here is the extracted epidemiological data:\n\n- **Reproduction number**: \n  - Overall reproduction number: 2.3 (1.6–3.2) in March\n  - Community reproduction number: 1.4 (0.9–2.2) in March, dropped to 0.7 (0.5–0.8) after April\n  - Hospital reproduction number: reduced to ≤0.1 after infection control measures\n\n- **Growth rate**: Not explicitly stated, but the text mentions that there was no exponential growth in incidence in Conakry.\n\n- **Dispersion parameter**: Not explicitly mentioned in the text.\n\n- **Pathogen Transmission**: Transmission occurs by direct contact with body fluids from symptomatic patients, care of patients at hospitals, and touching bodies at funerals.\n\n- **Zoonotic/primary infection**: The text does not specify if the infections were zoonotic or primary.\n\n- **Secondary transmission**: Secondary transmission occurred primarily in the community and between family members.\n\n- **Case-fatality ratio (CFR)**: Approximate

In [1]:
#zero-shot extraction with prompt engineering
import os
import pdfplumber
from langchain_openai import ChatOpenAI
from openai import OpenAI

api_key = os.getenv("OPENAI_API_KEY")
# 从PDF文件中提取文本
def extract_text_from_pdf(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text

# 使用模型直接提取信息
def extract_info_with_prompting(text):
    llm = ChatOpenAI(temperature=0, model="gpt-4o-mini", openai_api_key=api_key)
    
    prompt = f"""
    I will ask you several questions regarding epidemiological data from the following text.
    Please extract the required data points. If a specific piece of data is not present in the text, simply respond with "Data not found". If you are unsure about the data, respond with "Not certain".
    
    For each piece of data, provide a confidence level (High, Medium, Low).

    Extract the following epidemiological data:
    - Reproduction number
    - Growth rate
    - Dispersion parameter
    - Pathogen Transmission
    - Zoonotic/primary infection
    - Secondary transmission
    - Case-fatality ratio (CFR)
    - Infection-fatality ratio (IFR)
    - Attack rate
    - Symptom onset date
    - Onset-admission delay
    - Nosocomial infection
    - Onset-hospitalisation delay
    - Onset-death delay
    - Incubation period
    - Generation time
    - Serial interval
    - Infectious period
    - Clearance rate
    - Recovery rate
    - Transmission chain
    - Seroprevalence
    - Susceptibles
    - Linelist
    - Pandemic
    - Epidemic
    - Outbreak

    Text:
    {text}

    Format your response as:
    - [Data type]: [extracted data or "Data not found"]
    - Confidence level: [confidence level]
    """

    response = llm(prompt)
    return response

# 主函数
def main(pdf_path):
    # 提取PDF文本
    text = extract_text_from_pdf(pdf_path)
    
    # 使用prompting提取信息
    extracted_info = extract_info_with_prompting(text)
    
    # 显示提取的信息
    print(extracted_info)

# 调用主函数，传入PDF文件路径
pdf_path = 'Chains-of-transmission-and-control-of-Ebola-virus-.pdf'

main(pdf_path)


/Users/yutianhuan/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


content='- Reproduction number: 2.3 (1.6–3.2)\n- Confidence level: High\n\n- Growth rate: Data not found\n- Confidence level: Data not found\n\n- Dispersion parameter: Data not found\n- Confidence level: Data not found\n\n- Pathogen Transmission: Direct contact with body fluids from symptomatic patients\n- Confidence level: High\n\n- Zoonotic/primary infection: Data not found\n- Confidence level: Data not found\n\n- Secondary transmission: Data not found\n- Confidence level: Data not found\n\n- Case-fatality ratio (CFR): 70%\n- Confidence level: High\n\n- Infection-fatality ratio (IFR): Data not found\n- Confidence level: Data not found\n\n- Attack rate: Data not found\n- Confidence level: Data not found\n\n- Symptom onset date: Data not found\n- Confidence level: Data not found\n\n- Onset-admission delay: 5.0 days (SD 3.9)\n- Confidence level: High\n\n- Nosocomial infection: Yes, reported cases\n- Confidence level: High\n\n- Onset-hospitalisation delay: 5.0 days (SD 3.9)\n- Confidence

In [12]:
#Extract with Langchain 
import os
import pdfplumber
import pandas as pd
from langchain_openai import ChatOpenAI
from langchain.chains import create_extraction_chain

# 设置你的OpenAI API Key
api_key = os.getenv("OPENAI_API_KEY")

if api_key is None:
    raise ValueError("Please set the OPENAI_API_KEY environment variable")

# 定义架构
schema = {
    "properties": {
        "Reproduction number": {"type": "string"},
        "Growth rate": {"type": "string"},
        "Dispersion parameter": {"type": "string"},
        "Pathogen Transmission": {"type": "string"},
        "Zoonotic/primary infection": {"type": "string"},
        "Secondary transmission": {"type": "string"},
        "Case-fatality ratio (CFR)": {"type": "string"},
        "Infection-fatality ratio (IFR)": {"type": "string"},
        "Attack rate": {"type": "string"},
        "Symptom onset date": {"type": "string"},
        "Onset-admission delay": {"type": "string"},
        "Nosocomial infection": {"type": "string"},
        "Onset-hospitalisation delay": {"type": "string"},
        "Onset-death delay": {"type": "string"},
        "Incubation period": {"type": "string"},
        "Generation time": {"type": "string"},
        "Serial interval": {"type": "string"},
        "Infectious period": {"type": "string"},
        "Clearance rate": {"type": "string"},
        "Recovery rate": {"type": "string"},
        "Transmission chain": {"type": "string"},
        "Seroprevalence": {"type": "string"},
        "Susceptibles": {"type": "string"},
        "Linelist": {"type": "string"},
        "Pandemic": {"type": "string"},
        "Epidemic": {"type": "string"},
        "Outbreak": {"type": "string"}
    }
}

# 创建信息提取链
llm = ChatOpenAI(temperature=0, model="gpt-4o-mini", openai_api_key=api_key)
chain = create_extraction_chain(schema, llm)

# 从PDF文件中提取文本
def extract_text_from_pdf(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text

# 提取信息并创建数据框
def extract_info_from_text(text):
    response = chain.run(text)
    
    # 打印模型的响应
    print("Raw response from model:", response)    
    extracted_data = response[0]
    return pd.Series(extracted_data)

# 主函数
def main(pdf_path):
    # 提取PDF文本
    text = extract_text_from_pdf(pdf_path)
    
    # 提取信息
    extracted_series = extract_info_from_text(text)
    
    # 创建数据框
    df = pd.DataFrame([extracted_series])
    
    # 显示数据框
    print(df)

# 调用主函数，传入PDF文件路径
pdf_path = 'Chains-of-transmission-and-control-of-Ebola-virus-.pdf'
main(pdf_path)


Raw response from model: [{'Reproduction number': '2.3 (1.6–3.2)', 'Growth rate': 'not specified', 'Dispersion parameter': 'not specified', 'Pathogen Transmission': 'direct contact with body fluids from symptomatic patients', 'Zoonotic/primary infection': 'not specified', 'Secondary transmission': 'not specified', 'Case-fatality ratio (CFR)': '70%', 'Infection-fatality ratio (IFR)': 'not specified', 'Attack rate': 'not specified', 'Symptom onset date': 'not specified', 'Onset-admission delay': '5.0 (3.9) days', 'Nosocomial infection': 'yes', 'Onset-hospitalisation delay': 'not specified', 'Onset-death delay': '8.9 (4.0) days', 'Incubation period': '9.9 (9.0–11.0) days', 'Generation time': 'not specified', 'Serial interval': '14.2 days (13.1–15.5)', 'Infectious period': 'not specified', 'Clearance rate': 'not specified', 'Recovery rate': 'not specified', 'Transmission chain': 'yes', 'Seroprevalence': 'not specified', 'Susceptibles': 'not specified', 'Linelist': 'yes', 'Pandemic': 'not s

In [18]:
#Extract with Langchain and prompt engineering
import os
import pdfplumber
import pandas as pd
from langchain_openai import ChatOpenAI
from langchain.chains import create_extraction_chain

# 设置你的OpenAI API Key
api_key = os.getenv("OPENAI_API_KEY")

if api_key is None:
    raise ValueError("Please set the OPENAI_API_KEY environment variable")

# 定义架构
schema = {
    "properties": {
        "Reproduction number": {"type": "string"},
        "Growth rate": {"type": "string"},
        "Dispersion parameter": {"type": "string"},
        "Pathogen Transmission": {"type": "string"},
        "Zoonotic/primary infection": {"type": "string"},
        "Secondary transmission": {"type": "string"},
        "Case-fatality ratio (CFR)": {"type": "string"},
        "Infection-fatality ratio (IFR)": {"type": "string"},
        "Attack rate": {"type": "string"},
        "Symptom onset date": {"type": "string"},
        "Onset-admission delay": {"type": "string"},
        "Nosocomial infection": {"type": "string"},
        "Onset-hospitalisation delay": {"type": "string"},
        "Onset-death delay": {"type": "string"},
        "Incubation period": {"type": "string"},
        "Generation time": {"type": "string"},
        "Serial interval": {"type": "string"},
        "Infectious period": {"type": "string"},
        "Clearance rate": {"type": "string"},
        "Recovery rate": {"type": "string"},
        "Transmission chain": {"type": "string"},
        "Seroprevalence": {"type": "string"},
        "Susceptibles": {"type": "string"},
        "Linelist": {"type": "string"},
        "Pandemic": {"type": "string"},
        "Epidemic": {"type": "string"},
        "Outbreak": {"type": "string"}
    }
}

# 创建信息提取链
llm = ChatOpenAI(temperature=0, model="gpt-4o-mini", openai_api_key=api_key)
chain = create_extraction_chain(schema, llm)

# 从PDF文件中提取文本
def extract_text_from_pdf(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text

# 提取信息并创建数据框
def extract_info_from_text(text):
    # 定义自定义提示词
    custom_prompt = (
        "Extract the following information from the text below:"
        "\n\nBased on the following structure:"
        "\nReproduction number: {Reproduction number}"
        "\nGrowth rate: {Growth rate}"
        "\nDispersion parameter: {Dispersion parameter}"
        "\nPathogen Transmission: {Pathogen Transmission}"
        "\nZoonotic/primary infection: {Zoonotic_primary_infection}"
        "\nSecondary transmission: {Secondary_transmission}"
        "\nCase-fatality ratio (CFR): {Case_fatality_ratio}"
        "\nInfection-fatality ratio (IFR): {Infection_fatality_ratio}"
        "\nAttack rate: {Attack_rate}"
        "\nSymptom onset date: {Symptom_onset_date}"
        "\nOnset-admission delay: {Onset_admission_delay}"
        "\nNosocomial infection: {Nosocomial_infection}"
        "\nOnset-hospitalisation delay: {Onset_hospitalisation_delay}"
        "\nOnset-death delay: {Onset_death_delay}"
        "\nIncubation period: {Incubation_period}"
        "\nGeneration time: {Generation_time}"
        "\nSerial interval: {Serial_interval}"
        "\nInfectious period: {Infectious_period}"
        "\nClearance rate: {Clearance_rate}"
        "\nRecovery rate: {Recovery_rate}"
        "\nTransmission chain: {Transmission_chain}"
        "\nSeroprevalence: {Seroprevalence}"
        "\nSusceptibles: {Susceptibles}"
        "\nLinelist: {Linelist}"
        "\nPandemic: {Pandemic}"
        "\nEpidemic: {Epidemic}"
        "\nOutbreak: {Outbreak}"
        "\nText:"+text
    )
    
    response = chain.run(custom_prompt)
    
    # 打印模型的响应
    print("Raw response from model:", response)    
    extracted_data = response[0]
    return pd.Series(extracted_data)


# 主函数
def main(pdf_path):
    # 提取PDF文本
    text = extract_text_from_pdf(pdf_path)
    
    # 提取信息
    extracted_series = extract_info_from_text(text)
    
    # 创建数据框
    df = pd.DataFrame([extracted_series])
    
    # 显示数据框
    print(df)

# 调用主函数，传入PDF文件路径
pdf_path = 'Chains-of-transmission-and-control-of-Ebola-virus-.pdf'
main(pdf_path)

Raw response from model: [{'Reproduction number': '2.3 (1.6–3.2)', 'Growth rate': 'not specified', 'Dispersion parameter': 'not specified', 'Pathogen Transmission': 'direct contact with body fluids from symptomatic patients', 'Zoonotic/primary infection': 'not specified', 'Secondary transmission': 'not specified', 'Case-fatality ratio (CFR)': '70%', 'Infection-fatality ratio (IFR)': 'not specified', 'Attack rate': 'not specified', 'Symptom onset date': 'not specified', 'Onset-admission delay': '5.0 days (SD 3.9)', 'Nosocomial infection': 'yes', 'Onset-hospitalisation delay': 'not specified', 'Onset-death delay': '8.9 days (4.0)', 'Incubation period': '9.9 (9.0–11.0) days', 'Generation time': 'not specified', 'Serial interval': '14.2 days (13.1–15.5)', 'Infectious period': 'not specified', 'Clearance rate': 'not specified', 'Recovery rate': 'not specified', 'Transmission chain': 'yes', 'Seroprevalence': 'not specified', 'Susceptibles': 'not specified', 'Linelist': 'yes', 'Pandemic': 'no

In [3]:
#Pydantic
import os
import pdfplumber
import pandas as pd
from typing import Optional
from langchain_openai import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from pydantic import BaseModel, ValidationError

# 设置你的OpenAI API Key
api_key = os.getenv("OPENAI_API_KEY")

if api_key is None:
    raise ValueError("Please set the OPENAI_API_KEY environment variable")

# Pydantic model for epidemiological data
class EpidemiologicalData(BaseModel):
    reproduction_number: Optional[str]
    growth_rate: Optional[str]
    dispersion_parameter: Optional[str]
    pathogen_transmission: Optional[str]
    zoonotic_primary_infection: Optional[str]
    secondary_transmission: Optional[str]
    case_fatality_ratio: Optional[str]
    infection_fatality_ratio: Optional[str]
    attack_rate: Optional[str]
    symptom_onset_date: Optional[str]
    onset_admission_delay: Optional[str]
    nosocomial_infection: Optional[str]
    onset_hospitalisation_delay: Optional[str]
    onset_death_delay: Optional[str]
    incubation_period: Optional[str]
    generation_time: Optional[str]
    serial_interval: Optional[str]
    infectious_period: Optional[str]
    clearance_rate: Optional[str]
    recovery_rate: Optional[str]
    transmission_chain: Optional[str]
    seroprevalence: Optional[str]
    susceptibles: Optional[str]
    linelist: Optional[str]
    pandemic: Optional[str]
    epidemic: Optional[str]
    outbreak: Optional[str]

# Set up a Pydantic parser and prompt template
parser = PydanticOutputParser(pydantic_object=EpidemiologicalData)
prompt = PromptTemplate(
    template="Extract the following epidemiological data from the given text:\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# 从PDF文件中提取文本
def extract_text_from_pdf(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            if page.extract_text():
                text += page.extract_text() + "\n"
    return text

# 使用模型直接提取信息
def extract_info_with_prompting(text):
    model = ChatOpenAI(temperature=0, model="gpt-4o-mini", openai_api_key=api_key)
    _input = prompt.format(query=text)
    response = model(_input)
    
    # 调试输出
    print("Type of response:", type(response))
    print("Response from model:", response)
    
    try:
        response_text = response.content  # 直接访问AIMessage对象的内容
        parsed_response = parser.parse(response_text)
        return parsed_response.model_dump()
    except ValidationError as e:
        print("Validation error:", e)
        return {}
    except Exception as e:
        print("Error:", e)
        return {}

# 处理PDF文件并提取数据
def process_pdf(pdf_path):
    # 提取PDF文本
    text = extract_text_from_pdf(pdf_path)
    
    # 使用prompting提取信息
    extracted_info = extract_info_with_prompting(text)
    
    # 将提取的信息转换为DataFrame
    flat_data = {key: [value] for key, value in extracted_info.items()}
    df_epi = pd.DataFrame(flat_data)
    
    return df_epi

# 调用主函数，传入PDF文件路径
pdf_path = 'Chains-of-transmission-and-control-of-Ebola-virus-.pdf'
intel_df = process_pdf(pdf_path)

# 显示提取的信息
print(intel_df.head())



Type of response: <class 'langchain_core.messages.ai.AIMessage'>
Response from model: content='```json\n{\n  "reproduction_number": "2.3 (1.6–3.2)",\n  "growth_rate": "not specified",\n  "dispersion_parameter": "not specified",\n  "pathogen_transmission": "direct contact with body fluids from symptomatic patients",\n  "zoonotic_primary_infection": "not specified",\n  "secondary_transmission": "not specified",\n  "case_fatality_ratio": "70%",\n  "infection_fatality_ratio": "not specified",\n  "attack_rate": "not specified",\n  "symptom_onset_date": "not specified",\n  "onset_admission_delay": "5.0 (3.9)",\n  "nosocomial_infection": "not specified",\n  "onset_hospitalisation_delay": "not specified",\n  "onset_death_delay": "8.9 (4.0)",\n  "incubation_period": "9.9 (9.0–11.0)",\n  "generation_time": "not specified",\n  "serial_interval": "14.2 (13.1–15.5)",\n  "infectious_period": "not specified",\n  "clearance_rate": "not specified",\n  "recovery_rate": "not specified",\n  "transmission_

In [22]:
#Pydantic with prompt engineering
import os
import pdfplumber
import pandas as pd
from typing import Optional
from langchain_openai import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from pydantic import BaseModel, ValidationError

# 设置你的OpenAI API Key
api_key = os.getenv("OPENAI_API_KEY")

if api_key is None:
    raise ValueError("Please set the OPENAI_API_KEY environment variable")

# Pydantic model for epidemiological data
class EpidemiologicalData(BaseModel):
    reproduction_number: Optional[str]
    growth_rate: Optional[str]
    dispersion_parameter: Optional[str]
    pathogen_transmission: Optional[str]
    zoonotic_primary_infection: Optional[str]
    secondary_transmission: Optional[str]
    case_fatality_ratio: Optional[str]
    infection_fatality_ratio: Optional[str]
    attack_rate: Optional[str]
    symptom_onset_date: Optional[str]
    onset_admission_delay: Optional[str]
    nosocomial_infection: Optional[str]
    onset_hospitalisation_delay: Optional[str]
    onset_death_delay: Optional[str]
    incubation_period: Optional[str]
    generation_time: Optional[str]
    serial_interval: Optional[str]
    infectious_period: Optional[str]
    clearance_rate: Optional[str]
    recovery_rate: Optional[str]
    transmission_chain: Optional[str]
    seroprevalence: Optional[str]
    susceptibles: Optional[str]
    linelist: Optional[str]
    pandemic: Optional[str]
    epidemic: Optional[str]
    outbreak: Optional[str]

# Set up a Pydantic parser and prompt template
parser = PydanticOutputParser(pydantic_object=EpidemiologicalData)
prompt = PromptTemplate(
    template="""
    You are an expert in epidemiology and are tasked with extracting key epidemiological data from a given text.
    Please extract the relevant epidemiological parameters as specified by the following schema.
    If any data is not found or if you are uncertain, indicate with "Data not found" or "Not certain". 
    Provide each data point along with your confidence level. Make sure response for all of the parameters.

    Text:
    {query}

    Format your response as a JSON object matching the schema below:
    {format_instructions}
    """,
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# 从PDF文件中提取文本
def extract_text_from_pdf(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            if page.extract_text():
                text += page.extract_text() + "\n"
    return text

#提取数据
def extract_info_with_prompting(text):
    model = ChatOpenAI(temperature=0, model="gpt-4o-mini", openai_api_key=api_key)
    _input = prompt.format(query=text)
    response = model(_input)
    
    # 调试输出
    print("Type of response:", type(response))
    print("Response from model:", response)
    
    try:
        response_text = response.content  # 直接访问AIMessage对象的内容
        parsed_response = parser.parse(response_text)
        return parsed_response.model_dump()
    except ValidationError as e:
        print("Validation error:", e)
        return {}
    except Exception as e:
        print("Error:", e)
        return {}
    
# 处理PDF文件并提取数据
def process_pdf(pdf_path):
    # 提取PDF文本
    text = extract_text_from_pdf(pdf_path)
    
    # 使用prompting提取信息
    extracted_info = extract_info_with_prompting(text)
    
    # 将提取的信息转换为DataFrame
    flat_data = {key: [value] for key, value in extracted_info.items()}
    df_epi = pd.DataFrame(flat_data)
    
    return df_epi

# 调用主函数，传入PDF文件路径
pdf_path = 'Chains-of-transmission-and-control-of-Ebola-virus-.pdf'
intel_df = process_pdf(pdf_path)

# 显示提取的信息
print(intel_df)


Type of response: <class 'langchain_core.messages.ai.AIMessage'>
Response from model: content='```json\n{\n  "reproduction_number": "2.3 (1.6–3.2)",\n  "growth_rate": "Not certain",\n  "dispersion_parameter": "Not certain",\n  "pathogen_transmission": "Direct contact with body fluids from symptomatic patients",\n  "zoonotic_primary_infection": "Data not found",\n  "secondary_transmission": "Mean of 2.3 people infected by individuals with Ebola virus disease",\n  "case_fatality_ratio": "70%",\n  "infection_fatality_ratio": "Not certain",\n  "attack_rate": "Data not found",\n  "symptom_onset_date": "Not certain",\n  "onset_admission_delay": "5.0 days (SD 3.9)",\n  "nosocomial_infection": "Health-care workers contributed little to transmission",\n  "onset_hospitalisation_delay": "Not certain",\n  "onset_death_delay": "8.9 days (4.0)",\n  "incubation_period": "9.9 days (9.0–11.0)",\n  "generation_time": "Not certain",\n  "serial_interval": "14.2 days (13.1–15.5)",\n  "infectious_period": "

In [21]:
#extract with pedantic and segment
import os
import pdfplumber
import pandas as pd
from typing import Optional
from langchain_openai import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from pydantic import BaseModel, ValidationError

# 设置你的OpenAI API Key
api_key = os.getenv("OPENAI_API_KEY")

if api_key is None:
    raise ValueError("Please set the OPENAI_API_KEY environment variable")

# Pydantic model for epidemiological data
class EpidemiologicalData(BaseModel):
    reproduction_number: Optional[str]
    growth_rate: Optional[str]
    dispersion_parameter: Optional[str]
    pathogen_transmission: Optional[str]
    zoonotic_primary_infection: Optional[str]
    secondary_transmission: Optional[str]
    case_fatality_ratio: Optional[str]
    infection_fatality_ratio: Optional[str]
    attack_rate: Optional[str]
    symptom_onset_date: Optional[str]
    onset_admission_delay: Optional[str]
    nosocomial_infection: Optional[str]
    onset_hospitalisation_delay: Optional[str]
    onset_death_delay: Optional[str]
    incubation_period: Optional[str]
    generation_time: Optional[str]
    serial_interval: Optional[str]
    infectious_period: Optional[str]
    clearance_rate: Optional[str]
    recovery_rate: Optional[str]
    transmission_chain: Optional[str]
    seroprevalence: Optional[str]
    susceptibles: Optional[str]
    linelist: Optional[str]
    pandemic: Optional[str]
    epidemic: Optional[str]
    outbreak: Optional[str]

# Set up a Pydantic parser and prompt template
parser = PydanticOutputParser(pydantic_object=EpidemiologicalData)
prompt = PromptTemplate(
    template="""
    You are an expert in epidemiology and are tasked with extracting key epidemiological data from a given text.
    Please extract the relevant epidemiological parameters as specified by the following schema.
    If any data is not found or if you are uncertain, indicate with "Data not found" or "Not certain". 
    Provide each data point along with your confidence level. Make sure response for all of the parameters.

    Text:
    {query}

    Format your response as a JSON object matching the schema below:
    {format_instructions}
    """,
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# 从PDF文件中提取文本
def extract_text_from_pdf(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            if page.extract_text():
                text += page.extract_text() + "\n"
    return text

# 使用模型直接提取信息
def extract_info_with_prompting(text):
    model = ChatOpenAI(temperature=0, model="gpt-4o-mini", openai_api_key=api_key)
    text_segments = [text[i:i+1500] for i in range(0, len(text), 1500)]  # 分段处理
    previous_response = {}

    for segment in text_segments:
        _input = prompt.format(query=segment)
        response = model(_input)

        try:
            response_text = response.content
            parsed_response = parser.parse(response_text)
            current_response = parsed_response.model_dump()

            # 合并当前段落的结果与之前的结果
            for key, value in current_response.items():
                if value and value != "Data not found" and value != "Not certain":
                    previous_response[key] = value
            
        except ValidationError as e:
            print("Validation error:", e)
        except Exception as e:
            print("Error:", e)
    
    print("Model raw response:", previous_response)
    return previous_response
    

# 处理PDF文件并提取数据
def process_pdf(pdf_path):
    # 提取PDF文本
    text = extract_text_from_pdf(pdf_path)
    
    # 使用prompting提取信息
    extracted_info = extract_info_with_prompting(text)
    
    # 将提取的信息转换为DataFrame
    flat_data = {key: [value] for key, value in extracted_info.items()}
    df_epi = pd.DataFrame(flat_data)
    
    return df_epi

# 调用主函数，传入PDF文件路径
pdf_path = 'Chains-of-transmission-and-control-of-Ebola-virus-.pdf'
intel_df = process_pdf(pdf_path)

# 显示提取的信息
print(intel_df)


Model raw response: {'epidemic': 'manageable size with less than ten cases per week', 'reproduction_number': '≤1', 'pathogen_transmission': 'Ebola virus disease', 'nosocomial_infection': 'Increased risk among health-care workers', 'transmission_chain': 'Large chain of transmission in Conakry', 'secondary_transmission': 'high viraemia infected more people', 'case_fatality_ratio': '54% (95% CI 49–63)', 'incubation_period': '9.9 days (95% CI 9.0–11.0)', 'linelist': '20 cases', 'onset_admission_delay': '5.0 days (SD 3.9)', 'onset_death_delay': '8.9 days (4.0)', 'attack_rate': '35% (7 of 20) and 15% (3) in March', 'growth_rate': '0.4–1.1', 'outbreak': 'Ebola virus disease', 'serial_interval': '14.2 days (95% CI 13.1–15.5)'}
                                            epidemic reproduction_number  \
0  manageable size with less than ten cases per week                  ≤1   

  pathogen_transmission                      nosocomial_infection  \
0   Ebola virus disease  Increased risk among hea

In [17]:
import os
import pdfplumber
from langchain_openai import ChatOpenAI
from openai import OpenAI

api_key = os.getenv("OPENAI_API_KEY")

def extract_text_and_tables(pdf_path):
    combined_text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            # 提取普通文本
            page_text = page.extract_text()
            if page_text:
                combined_text += page_text + "\n"
            
            # 提取表格内容并格式化
            tables = page.extract_tables()
            for table in tables:
                table_text = "\n".join(["\t".join([str(cell) if cell is not None else '' for cell in row]) for row in table if row])
                combined_text += table_text + "\n"
    
    return combined_text

# 使用模型直接提取信息
def extract_info_with_prompting(text):
    llm = ChatOpenAI(temperature=0, model="gpt-4o-mini", openai_api_key=api_key)
    
    prompt = f"""
    Extract the following epidemiological data from the given text:
    - Reproduction number
    - Growth rate
    - Dispersion parameter
    - Pathogen Transmission
    - Zoonotic/primary infection
    - Secondary transmission
    - Case-fatality ratio (CFR)
    - Infection-fatality ratio (IFR)
    - Attack rate
    - Symptom onset date
    - Onset-admission delay
    - Nosocomial infection
    - Onset-hospitalisation delay
    - Onset-death delay
    - Incubation period
    - Generation time
    - Serial interval
    - Infectious period
    - Clearance rate
    - Recovery rate
    - Transmission chain
    - Seroprevalence
    - Susceptibles
    - Linelist
    - Pandemic
    - Epidemic
    - Outbreak

    Text:
    {text}
    """

    response = llm(prompt)
    return response

# 主函数
def main(pdf_path):
    # 提取PDF文本
    text = extract_text_and_tables(pdf_path)
    
    # 使用prompting提取信息
    extracted_info = extract_info_with_prompting(text)
    
    # 显示提取的信息
    print(extracted_info)

# 调用主函数，传入PDF文件路径
pdf_path = 'Chains-of-transmission-and-control-of-Ebola-virus-.pdf'

main(pdf_path)

content='Based on the provided text, here is the extracted epidemiological data:\n\n- **Reproduction number**: \n  - Overall reproduction number: 2.3 (1.6–3.2) in March\n  - Community reproduction number: 1.4 (0.9–2.2) in March, dropped to 0.7 (0.5–0.8) after April for admitted patients\n  - Hospital reproduction number: ≤0.1 after infection control measures\n\n- **Growth rate**: Not explicitly stated, but the text mentions that there was no exponential growth in incidence in Conakry.\n\n- **Dispersion parameter**: Not explicitly mentioned in the text.\n\n- **Pathogen Transmission**: Transmission occurs by direct contact with body fluids from symptomatic patients, care of patients at hospitals, and touching bodies at funerals.\n\n- **Zoonotic/primary infection**: The text does not specify if the infections were zoonotic or primary.\n\n- **Secondary transmission**: The text discusses secondary cases but does not provide a specific number.\n\n- **Case-fatality ratio (CFR)**: Approximatel